In [29]:
import pickle
import pandas as pd
import jams
import sqlite3
import os
import sys

In [4]:
from tqdm import tqdm

In [5]:
def load_cf_targets():
    data = pickle.load(open('cf_target_data.pickle', 'rb'))

    data.keys()


    model = data['best_model']
    item_encoder = data['ix']
    song_to_track = data['song_to_track']

    return model, item_encoder, song_to_track

In [15]:
metadata = {}
with sqlite3.connect('/home/bmcfee/data/1mil/AdditionalFiles/track_metadata.db') as dbc:

    cur = dbc.cursor()
    for meta in tqdm(cur.execute("""SELECT track_id, song_id, artist_id, artist_name, title, duration from songs""")):
        meta = list(meta)
        track_id = meta.pop(0)
        metadata[track_id] = dict(zip(['song_id', 'artist_id', 'artist', 'title', 'duration'], meta))

metadata = pd.DataFrame.from_dict(metadata, orient='index')
metadata.index.name = 'track_id'

In [73]:
def construct_dir(target_dir, track_id):
    
    crumbs = [target_dir, 'jams']
    crumbs.extend(track_id[2:5])
    
    dirname = os.path.sep.join(crumbs)
    
    jams.util.smkdirs(dirname)
    return dirname
    
def vec_ser(vec):
    
    return [float(_) for _ in vec]
    
def create_jam(target_dir, track_id, meta, vec, ext='jamz'):
    
    outdir = construct_dir(target_dir, track_id)
    
    jam = jams.JAMS()
    jam.file_metadata.artist = meta['artist']
    jam.file_metadata.title = meta['title']
    jam.file_metadata.duration = float(meta['duration'])
    
    jam.file_metadata.identifiers.artist_id = meta['artist_id']
    
    ann = jams.Annotation(namespace='vector')
    
    ann.append(time=0.0, duration=float(meta['duration']), value=vec_ser(vec))
    jam.annotations.append(ann)
    
    jam.save(os.path.sep.join([outdir, os.path.extsep.join([track_id, ext])]))

In [6]:
model, item_encoder, song_to_track = load_cf_targets()

In [30]:
TARGET_DIR = '/home/bmcfee/data/1mil/'

In [76]:
for i in tqdm(range(len(model.item_embeddings))):
    vec = model.item_embeddings[i]
    song_id = item_encoder.inverse_transform(i)
    track_id = song_to_track[song_id]
    meta = dict(metadata.loc[track_id])
    
    create_jam(TARGET_DIR, track_id, meta, vec)